### CS/ECE/ISyE 524 &mdash; Introduction to Optimization &mdash; Summer 2025 ###

# Optimized Meal Planning #

#### Keiji Toriumi (ktoriumi@wisc.edu), Myles Carpenter (mmcarpenter@wisc.edu)

### Table of Contents

1. [Introduction](#1.-Introduction)
1. [Mathematical Model](#2.-Mathematical-model)
1. [Solution](#3.-Solution)
1. [Results and Discussion](#4.-Results-and-discussion)
1. [Optional Subsection](#4.A.-Feel-free-to-add-subsections)
1. [Conclusion](#5.-Conclusion)
1. [References](#6.-References)

## 1. Introduction ##

Meal planning is a widespread issue for people all over the world. It is a difficult balance of getting enough nutrients, not wasting food, and spending as little money as possible. As someone planning how to buy ingredients for weekly grocery shopping, the question is what ingredients do you need to buy and what recipes should you make.

Given a set of recipes that you know and a set of ingredients that are available to buy from the grocery store, how can grocery shopping be optimized to save money. Given that a kitchen stores food from the past, constraints involving the amount of food that can be stored and how long it can be stored must be considered for a realistic model. A proper grocery shopping plan will also cover all necessary nutrients in a given week.

Important progress in this problem has been made in the past. George Stigler, an American economist, addressed the issue of how to minimize the cost of fulfilling all of a person's nutrition requirements given a list of food with known nutritional values (Stigler, 1945). This work was one of the earliest examples of an optimization problem. Stigler used a heuristic approach but developed the foundation for a problem that is solvable using modern optimization. This work is useful for people that go grocery shopping and wish to minimize the cost. It makes planning easier and more efficient.

Other researchers have modernized Stigler's work. Modern datasets and linear programming techniques gave more precise and thorough results (Garille, 2001). They have also extended the applicability of the data by performing the optimization with different circumstances for nutrition requirements. 

This work builds off of past work in fulfilling nutrition requirements using optimization techniques but also allows for meal planning across weeks, knowing that food can be stored in the fridge after being bought.

The model in this program will use a mix of data types. This model is highly customizable. Any recipes can be chosen but a set of recipes will be created by hand for the purpose of demonstration. Recipes will consist of fixed ratios of each ingredient in the database. These recipes will use real ingredients. The nutrition information for each ingredient can be found online (https://www.nutritionix.com/search?q=co). As grocery prices fluctuate, prices will be chosen based on a snapshot of current prices as of xx-xx-xxxx from a typical Wisconsin grocery store website (https://www.metromarket.net/). Estimates for volume taken up by each ingredient will be made by hand.

## 2. Mathematical model ##

Data

$$
\begin{aligned}
T, \text{the set of discrete time periods} \\
I, \text{the set of ingredients that are available} \\
N, \text{the set of nutrients that are tracked} \\
R, \text{the set of known recipes} \\
F_{ij}, \text{the amount of nutrient } j \text{ in a unit amount of ingredient } i,\ \forall i \in I,\ j \in N \\
S_{j}, \text{the amount of nutrient } j \text{ that is needed each week for a healthy diet} \\
C_{i}, \text{the unit cost of ingredient } i,\ \forall i \in I \\
V_{i}, \text{the unit volume of ingredient } i,\ \forall i \in I \\
Fr, \text{the max volume of the fridge} \\
Req_{il}, \text{the amount of ingredient i needed for recipe} \ l \ \forall i \in I, l \in R \\
MAX, \text{the maximum number of recipes that can be made in a week} (tentative)
\end{aligned}
$$


Decision variables:
- $x_{ik}$, the amount of ingredient i bought in week k, $\forall i \in I, k \in T$
- $y_{lk}$, the amount of recipe $l$ to make in week k, $\forall l \in R, k \in T$
- $z_{ik}$, the amount of ingredient i to store in the fridge in week k, $\forall i \in I, k \in T$

Linear Program:
\begin{align}
\min \sum_{k \in T} \sum_{i \in I} x_{ik} * C_i \\
\text{s.t. } \sum_{l \in R} \sum_{i \in I} y_{lk} * Req_{il} * F_{ij} \geq S_{j}, \forall k \in T, j \in N \\
\sum_{i \in I} z_{ik} * V_{i} \leq Fr, \forall k \in T \\
z_{i0} = 0, \forall i \in I \\
x_{ik} + z_{i,k-1} = \sum_{l \in R} y_{lk} * Req_{il} + z_{ik}, \forall k \in T, i \in I \\
\sum_{l \in R} y_{lk} \leq MAX, \forall k \in T \\
y_{lk}, x_{ik}, z_{ik} \geq 0, \forall i \in I, k \in T
\end{align}

- The first constraint ensures there are enough nutrients
- The second constraint ensures that the fridge is not overfilled
- The third constraint sets the intial amount of food to 0
- The fourth constraint ensures that the amount of each ingredient being consumed is balanced by the amount being bought
- The fifth constraint limits the amount of recipes made in a week (i.e. only 7 recipes worth of recipes being made)

## 3. Solution ##

In [2]:
# for now leave out multi-period planning and general form - too complex

I = ["rice", "chicken nuggets", "steak"]
N = ["calories", "protein"]
R = ["steak and rice", "just nuggets"]
F = [300 0; 200 5; 400 30]
S = [2000, 100]
C = [5, 5, 15]
Req = [1 0; 0 1; 1 0]

# make the model
using JuMP, HiGHS

m = Model(HiGHS.Optimizer)

@variable(m, x[1:3] >= 0)
@variable(m, y[1:2] >= 0)

@constraint(m, nutr[j in 1:2], sum(y[l] * sum(Req[i,l] * F[i,j] for i in 1:3) for l in 1:2) >= S[j])
@constraint(m, init_balance[i in 1:3], x[i] >= sum( Req[i,l]*y[l] for l in 1:2 )) 

@objective(m, Min, sum( x[i] * C[i] for i in 1:3 ))

optimize!(m)

println("Buy ingredients ", value.(x))
println("Make recipes ", value.(y))
println("Total cost ", objective_value(m))

Running HiGHS 1.11.0 (git hash: 364c83a51e): Copyright (c) 2025 HiGHS under MIT licence terms
LP   has 5 rows; 5 cols; 10 nonzeros
Coefficient ranges:
  Matrix [1e+00, 7e+02]
  Cost   [5e+00, 2e+01]
  Bound  [0e+00, 0e+00]
  RHS    [1e+02, 2e+03]
Presolving model
2 rows, 2 cols, 4 nonzeros  0s
2 rows, 2 cols, 4 nonzeros  0s
Presolve : Reductions: rows 2(-3); columns 2(-3); elements 4(-6)
Solving the presolved LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0     0.0000000000e+00 Pr: 2(900) 0s
          3     6.6666666667e+01 Pr: 0(0) 0s
Solving the original LP from the solution after postsolve
Model status        : Optimal
Simplex   iterations: 3
Objective value     :  6.6666666667e+01
P-D objective error :  1.0578800036e-16
HiGHS run time      :          0.00
[3.3333333333333335, -0.0, 3.3333333333333335]
Make recipes [3.3333333333333335, 0.0]
Total cost 66.66666666666667


## 4. Results and discussion ##

So far, a multiperiod planning linear program has been written. It includes descriptions of the data and variables that will be used. A basic linear program has been implemented in Julia that optimizes ingredient selection for a single week using known recipes. It fulfills nutrition requirements and minimizes cost. It is a basic model that can be added on to create the full multiperiod planning model. 

The remaining tasks to be done and rough timelines:
- Upgrade the Julia code to function as a multiperiod planning linear program (7/30)
- Decide on how to incentivize storing food (7/31) 
- Decide on further improvements to the mode (integer constraints, MCNF, etc.)l (8/02)
- Create full dataset for the program (8/03)

### 4.A. Feel free to add subsections

**Issues/Concerns**
1. Many of the variables should be integers. Many ingredients can only be bought from a store in fixed amounts, thought ingredients can be kept as a fractional amount in the fridge/pantry. Recipes can be difficult to scale by fractional amounts, though scaling recipes by an integer is always straightforward. Two approaches that would help with these concerns are integer programming techniques and formulating the problem as a Min-Cost Network Flow Problem with unimodular data matrices.
2. It may make sense to add a tradeoff variable. The MAX recipes per week variable is a constraint that could be itself optimized. Same thing for the recipes. Substituting ingredients and tweaking the proportions are realistic things a cook do to account for dietary restrictions, ingredient scarcity, flavor preference, etc.
3. Ingredient costs may fluctuate over time. It may make sense to further complicate the data and change $C_i$ to $C_{ik} \ (i \in I, k \in T)$
4. Assuming 3 is implemented, it is not realistic to know every future price fluctuation in advance. Therefore it makes more sense to create a model that is blind to future conditions.
5. There are conceivable situations where there is no solutions(for example, a vegan may struggle to reach protien nutrient constraint if there is a lack of special ingredients in the market)

**Possible Approaches for concerns**
1. The next couple weeks of lecture may enlighten us on new techniques to approach problems with an integer constraint. It will take some mental work to re-imagine the problem as combinatorial, but that seems like a goal that can be reasonable achieved by the deadline. The fact that inventory can be stored in fractional amounts is already accounted for in the existing model.

2. This seems like less of a priority for our project than other issues, but I think this would be useful for an end user using our model.

3. As we move on to generating data with scripts rather that hand-typing, we will see how cost-change over time makes data-creation more complex.

4. This seems like a possible application of stochastic programming, which I know almost nothing about and may be beyond the scope of this class. I wonder if there is a way to force the model to only consider only the next $t \leq |T|$ time slices, using only constraints.

5. We can use least-squares techniques to find the diet that gets someone as close to the nutrient requirements as possible.
#### 4.A.a. or subsubsections

## 6. Conclusion ##

Summarize your findings and your results, and talk about at least one possible future direction; something that might be interesting to pursue as a follow-up to your project.

## 7. References ##

George J. Stigler, (1945), Journal of Farm Economics. Vol. 27, No. 2, pp. 303-314, Oxford University Press
https://doi.org/10.2307/1231810

Susan Garner Garille, Saul I. Gass, (2001) Stigler's Diet Problem Revisited. Operations Research 49(1):1-13.
https://doi.org/10.1287/opre.49.1.1.11187